In [ ]:
# Dependencies
from sodapy import Socrata
import json
import requests
import pandas as pd
from config import app_token

In [ ]:
# Use open client to obtain data
client = Socrata("data.cdc.gov", app_token)
results = client.get("9bhg-hcku", limit=4000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

In [ ]:
results_states = results_df.loc[(results_df['state'] != 'United States') & (results_df['age_group'] == 'All Ages') & (results_df['group'] == 'By Total') & (results_df['sex'] == 'All Sexes')]
results_states = results_states[['state', 'covid_19_deaths']]
results_states

In [ ]:
results_sex = results_df.loc[((results_df['state'] == 'Texas') | (results_df['state'] == 'California')) & (results_df['age_group'] == 'All Ages') & (results_df['group'] == 'By Total') & (results_df['sex'] != 'All Sexes')]
results_sex = results_sex[['state', 'sex', 'covid_19_deaths']]
results_sex

In [ ]:
results_age = results_df.loc[((results_df['state'] == 'Texas') | (results_df['state'] == 'California')) & (results_df['age_group'] != 'All Ages') & (results_df['group'] == 'By Total') & (results_df['sex'] == 'All Sexes')]
results_age2 = results_age.loc[(results_df['age_group'] == '0-17 years') | (results_df['age_group'] == '18-29 years') | (results_df['age_group'] == '30-39 years') | (results_df['age_group'] == '40-49 years') | (results_df['age_group'] == '50-64 years') | (results_df['age_group'] == '65-74 years') | (results_df['age_group'] == '75-84 years') | (results_df['age_group'] == '85 years and over')]
results_age2 = results_age2[['state', 'age_group', 'covid_19_deaths']]
results_age2

In [ ]:
# Use open client to obtain data
results2 = client.get("ks3g-spdg", limit=4000000)

# Convert to pandas DataFrame
results2_df = pd.DataFrame.from_records(results2)
results2_df

In [ ]:
results_race = results2_df.loc[((results2_df['state'] == 'Texas') | (results2_df['state'] == 'California')) & (results2_df['race_and_hispanic_origin'] != 'Total Deaths')]
results_race = results_race.loc[(results_race['age_group_new'] == '0-17 years') | (results_race['age_group_new'] == '18-29 years') | (results_race['age_group_new'] == '30-39 years') | (results_race['age_group_new'] == '40-49 years') | (results_race['age_group_new'] == '50-64 years') | (results_race['age_group_new'] == '65-74 years') | (results_race['age_group_new'] == '75-84 years') | (results_race['age_group_new'] == '85 years and over')]
results_race2 = results_race2.fillna(0)

converted_race_df = results_race2.copy()
converted_race_df = converted_race_df.astype({"covid_19_deaths": "int64"})
converted_race_df.dtypes

group_race_df = converted_race_df.groupby(['state','race_and_hispanic_origin'])
group_race_df.dtypes

total_deaths_race_df = pd.DataFrame(group_race_df[["covid_19_deaths"]].sum())
total_deaths_race_df